# AlexNet 实现与测试

In [1]:
from datetime import datetime
import tensorflow as tf
import math
import time

In [2]:
batch_size =32
num_batches = 100

In [4]:
def print_activation(t):
    print(t.op.name, ' ', t.get_shape().as_list())

In [31]:
def inference(images):
    parameters = []
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11, 11, 3, 64], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), name='biases')
        conv1 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)
        parameters +=[kernel, biases]
        print_activation(conv1)
    lrn1 = tf.nn.lrn(conv1, 4, bias=1.0,  alpha=0.001/9, beta=0.75, name='lrn1')
    pool1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='pool1')
    print_activation(pool1)
    
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5, 5, 64, 192], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[192], dtype=tf.float32), trainable= True, name='biases')
        conv2 = tf.nn.relu(tf.nn.bias_add(conv, biases),  name=scope)
        parameters +=[kernel, biases]
        print_activation(conv2)
    lrn2 = tf.nn.lrn(conv2, 4, bias=1.0,  alpha=0.001/9, beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID', name='pool2')
    print_activation(pool2)
    
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 192, 384], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[384], dtype=tf.float32), trainable= True, name='biases')
        conv3 = tf.nn.relu(tf.nn.bias_add(conv, biases),  name=scope)
        parameters +=[kernel, biases]
        print_activation(conv3)
        
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 256], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable= True, name='biases')
        conv4 = tf.nn.relu(tf.nn.bias_add(conv, biases),  name=scope)
        parameters +=[kernel, biases]
        print_activation(conv4)

    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable= True, name='biases')
        conv5 = tf.nn.relu(tf.nn.bias_add(conv, biases),  name=scope)
        parameters +=[kernel, biases]
        print_activation(conv5)
        
    pool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool5')
    print_activation(pool5)
    return pool5, parameters
        

In [32]:
# 评估每轮计算时间
def time_tensorflow_run(session,  target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i>=num_steps_burn_in:
            if not i%10:
                print('%s: step %d, duration = %.3f'%(datetime.now(), i-num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration*duration
        
    mn = total_duration/num_batches
    vr = total_duration_squared/num_batches - mn*mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec/batch'%(datetime.now(), info_string, num_batches, mn, sd))
        

In [35]:
def  run_benchmark():
    g = tf.Graph()
    with g.as_default() :
        image_size =  224
        images = tf.Variable(tf.random_normal([batch_size, image_size, image_size, 3], dtype=tf.float32, stddev=1e-1))
        pool5, parameters =  inference(images)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        time_tensorflow_run(sess, pool5, "Forward")
        
        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective, parameters)
        time_tensorflow_run(sess, grad, "Forward-backward")

In [36]:
run_benchmark()

conv1   [32, 56, 56, 64]
pool1   [32, 27, 27, 64]
conv2   [32, 27, 27, 192]
pool2   [32, 13, 13, 192]
conv3   [32, 13, 13, 384]
conv4   [32, 13, 13, 256]
conv5   [32, 13, 13, 256]
pool5   [32, 6, 6, 256]
2017-03-25 12:34:50.704827: step 0, duration = 2.272
2017-03-25 12:35:15.617357: step 10, duration = 2.343
2017-03-25 12:35:40.375223: step 20, duration = 2.880
2017-03-25 12:36:05.096959: step 30, duration = 3.531
2017-03-25 12:36:31.346049: step 40, duration = 2.239
2017-03-25 12:36:56.085065: step 50, duration = 2.365
2017-03-25 12:37:20.550623: step 60, duration = 2.387
2017-03-25 12:37:45.084371: step 70, duration = 2.382
2017-03-25 12:38:11.976173: step 80, duration = 2.288
2017-03-25 12:38:39.488005: step 90, duration = 2.624
2017-03-25 12:39:02.711275: Forward across 100 steps, 2.543 +/- 0.325 sec/batch
2017-03-25 12:40:40.625715: step 0, duration = 10.037
2017-03-25 12:42:13.463385: step 10, duration = 9.842
2017-03-25 12:43:40.918737: step 20, duration = 8.085
2017-03-25 12:4